<b><p style="color:blue"> Getting started: PARAMETERS <p><b>

In [2]:
#Basic Parameters (API and URL)

api_key= "ea0518bbe4e05ba4b9685668b348a1d7"

api_private_key = "6704343727a4d01b047aec1033b8478f540932c9"

url_base = "http://gateway.marvel.com"


In [3]:
#Tailored Parameters (ts, hash)

import time

ts = 1000

ts_str = str(int(ts))

import hashlib

hash1 = str(ts_str)+api_private_key+api_key

hash2= hashlib.md5(hash1.encode())

hash3= hash2.hexdigest()

In [4]:
import requests #To be able to create API requests

<b>1. Provide a list of 30 Marvel characters</b>

In [5]:
endpoint_lists = "/v1/public/characters" #endpoint to retrieve characters

url_list = url_base + endpoint_lists #final url

#GET request for characters

body = {'limit':30, 'ts':ts_str, 'apikey': api_key, 'hash':hash3} #body parameters

characters = requests.get(url=url_list, params= body).json() #retrieve data of get request in json format

In [6]:
#Create a list with the characters
list_characters=[]
for i in characters['data']['results']:
    for j in i:
        if j == 'name':
            list_characters.append(i[j])

<b>2. Retrieve the Ids for all the characters in your list (in str form)</b>

In [7]:
#Create a list of IDs (we are using the same data from the character request)
#Instead of using the character name, we retrive the ID
list_ids=[]
for i in characters['data']['results']:
    for j in i:
        if j == 'id':
            list_ids.append(str(i[j]))

<b>3. Retrieve the total number of Events available for all the characters in your list (in integer form)</b>

In [8]:
body_events= { 'ts':ts_str,'apikey': api_key,'hash':hash3} #for the get method, we use it in parms
list_events = []
#To retrieve data we need to include the ID of each character in the endpoint
#Therefore, we use a loop that gets all the ID from the list_ids and place it in the endpoint
for i in list_ids:
    endpoint_events = "/v1/public/characters/{characterId}/events".format(characterId=int(i))
    url_final_events = url_base + endpoint_events
    c= requests.get(url=url_final_events, params= body_events).json()
    if c['data']['count'] == 0:
        list_events.append(None)
    else:
        list_events.append(int(c['data']['count']))

<b>4. Retrieve the total number of Series available for all the characters in your list  (in integer form)</b>

In [9]:
body_series= {'ts':ts_str,'apikey': api_key,'hash':hash3} #repeated from the previous, but for name conventionality, we assigned it to another variable
list_series = []
for i in list_ids:
    endpoint_events = "/v1/public/characters/{characterId}/series".format(characterId=int(i))
    url_final_series = url_base + endpoint_events
    c= requests.get(url=url_final_series, params= body_series).json()
    if c['data']['count'] == 0:
        list_series.append(None)
    else:
        list_series.append(int(c['data']['count']))          

<b>5. Retrieve the total number of Comics available for all the characters in your list (in integer form)</b>

In [55]:
body_comics= {'ts':ts_str,'apikey': api_key,'hash':hash3} #repeated from the previous, changed variable for convention
list_comics = []
for i in list_ids:
    endpoint_comics = "/v1/public/characters/{characterId}/comics".format(characterId=int(i))
    url_final_comics = url_base + endpoint_comics
    c= requests.get(url=url_final_comics, params= body_comics).json()
    if c['data']['count'] == 0:
        list_comics.append(None) 
    else:
        list_comics.append(c['data']['count'])

In [56]:
list_comics[2]

20

<b> 6. Retrieve the Price of the most expensive comic that the character was featured in or all the characters in your list (in float form and USD)</b>

In [74]:
body_prices= {'ts':ts_str,'apikey': api_key,'hash':hash3} #repeated from the previous, name convention
dic_prices= {} # we want to create a dictionary using id as keys and a list of prices as values
for i in list_ids:
    endpoint_comics = "/v1/public/characters/{characterId}/comics".format(characterId=int(i))
    url_final_comics = url_base + endpoint_comics
    c= requests.get(url=url_final_comics, params= body_comics).json()
    #Ensure to have None type if no results are found
    if len(c['data']['results'])==0:
        x= None
        dic_prices.setdefault(i, [])
        dic_prices[i].append(x)
    else:
        for j in c['data']['results']:#for each comic in the list
            for k in j['prices']:
                x= k['price']
                dic_prices.setdefault(i, []) #Create a different list considering the different IDs
                dic_prices[i].append(x)
# If the ids have no comics found, we provide a None type (ensuring the length of the final list is equal to all the other dataframe columns)

In [81]:
list_max_prices=[] 
#From the dictonary values we want to retrieve for every ID the max value
for x in dic_prices:
    i= max(dic_prices[x])
    if i==None:
        list_max_prices.append(None)
    else:
        list_max_prices.append('${:,.2f}'.format(i))

In [82]:
list_max_prices

['$2.99',
 '$2.99',
 '$125.00',
 '$2.99',
 '$75.00',
 '$3.99',
 '$75.00',
 '$9.99',
 '$9.99',
 None,
 '$125.00',
 None,
 '$4.99',
 '$49.99',
 '$15.99',
 '$9.99',
 '$3.95',
 '$39.99',
 None,
 '$9.99',
 '$2.99',
 None,
 None,
 None,
 None,
 '$2.99',
 '$6.99',
 '$3.99',
 None,
 '$2.99']

<b>7. Store the data above in a pandas DataFrame</b>

In [83]:
import pandas as pd

In [94]:
df = pd.DataFrame({'Character Name':list_characters, 'Character ID':list_ids,'Total Available Events':list_events,'Total Available Series':pd.Series(list_series,dtype=pd.Int64Dtype()),'Total Available Comics':pd.Series(list_comics,dtype=pd.Int64Dtype()),'Price of the Most Expensive Comic':list_max_prices })

In [95]:
df

,Character Name,Character ID,Total Available Events,Total Available Series,Total Available Comics,Price of the Most Expensive Comic
0,3-D Man,1011334,1.0,3,12,$2.99
1,A-Bomb (HAS),1017100,NaN,2,4,$2.99
2,A.I.M.,1009144,NaN,20,20,$125.00
3,Aaron Stack,1010699,NaN,3,14,$2.99
4,Abomination (Emil Blonsky),1009146,1.0,20,20,$75.00
5,Abomination (Ultimate),1016823,NaN,2,2,$3.99
6,Absorbing Man,1009148,4.0,20,20,$75.00
7,Abyss,1009149,1.0,3,8,$9.99
8,Abyss (Age of Apocalypse),1010903,1.0,3,3,$9.99
9,Adam Destine,1011266,NaN,NaN,NaN,None


In [73]:
print(df.dtypes)

Character Name                       object
Character ID                         object
Total Available Events                Int64
Total Available Series                Int64
Total Available Comics                Int64
Price of the Most Expensive Comic    object
dtype: object


In [25]:
df.to_csv(path_or_buf="data.csv")